# Integração com LLM

In [1]:
from openai import OpenAI

client = OpenAI(api_key="SUA_CHAVE_API")

def interpretar_comando(texto, contexto_visao):
    prompt = f"""
    Você é um assistente residencial. Um morador disse: "{texto}".
    A câmera detectou: {contexto_visao}.
    O que você deve fazer? Liste as ações no formato JSON.
    """
    resposta = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="gpt-4"
    )
    return resposta.choices[0].message.content
